In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pymc as pm
from pymc.Matplot import plot as pmplot


sns.set()


In [ ]:
dick = pd.read_pickle('country_data.pkl')

In [ ]:
US = pd.read_pickle('us_state_dict.pkl')
ny = US['New York']
ny = ny[['confirmed','deceased','recovered','inc','factor','dead_inc','dead_factor']]
dick['New York'] = ny

In [ ]:
country_name = 'US'

country = dick[country_name]

country

In [ ]:
def strip(x):
    return x.replace('\t','')

country_populations = pd.read_csv('world_pop.csv',
                                  sep=';',header=None,index_col=0,names=['population'],
                                 thousands=',',converters={0 : strip})



In [ ]:
country_posterior = pd.read_pickle('violin_daily_result_{}.pkl'.format(country_name.replace(' ','_')))
# r_post = growth factor confirmed, f_post = true mortality rate
country_posterior.describe()

In [ ]:
C = country['confirmed'][-1]

nr_samples = 1000

rows = np.random.choice(country_posterior.index,replace=True,size=nr_samples)
post_samples = country_posterior.loc[rows]


In [ ]:
X = np.linspace(0,len(country),nr_samples)
fwd_look = 40

forecast = np.zeros((fwd_look,nr_samples))

for fwd in range(fwd_look):
    sample_dead = np.array(
       [pm.rpoisson(C * post_samples.r_post.iloc[i] * post_samples.f_post.iloc[i]) for i in range(len(X))])
    
    forecast[fwd] = sample_dead

    # logistic function for daily growth of C : general form for growth factor : (1 - ( N  / population ) ) * N
    C  = C * country['factor'][-1] * (1 - (C / (country_populations.loc[country_name,'population'] \
                                                * country_posterior.f_post.mean() * 5)))
    

forecast_df = pd.DataFrame(forecast).T
forecast_df.index.name = 'sample'
forecast_df.columns.name = 'day'



In [ ]:
plt.figure(figsize=(18,12))

means = forecast_df.mean(axis=0)
current = country.iloc[-1,5]

over_current = means > current
forecast_df.drop(forecast_df.columns[over_current],axis=1,inplace=True)

print (forecast_df)
forecast_df.columns = range(0,len(forecast_df.columns) )

means = forecast_df.mean(axis=0)
CI_89 = np.percentile(forecast_df,[5.5,94.5],axis=0)
CI_89_low = CI_89[0,:]
CI_89_high = CI_89[1,:]


sns.violinplot(data=forecast_df,palette=['orange'],inner='box')

print (CI_89_low)
print (CI_89_high)

#start_day = country.index[0].date()
start_day = pd.datetime.today().date()


forecast_df.to_pickle('dead_forecast_{}_starting_{}.pkl'.format(country_name.replace(' ','_'),start_day))
forecast_df

In [ ]:
plt.figure(figsize=(18,12))
plt.errorbar(range(len(forecast_df.columns)),means,np.array((np.abs(means - CI_89_low),np.abs(means - CI_89_high))),fmt='o')